In [42]:
from __future__ import print_function
import numpy as np
import modern_robotics as mr

In [43]:
import numpy as np
import modern_robotics as mr

############
# In this block we calculate the dynamic model of the robot
# Having state(t) and actions (taw), it gives you state(t+1) 
############

def inertia_G(L_list,m_list, r_list=None):#works only for 2D
    n=len(L_list)
    G_list=np.empty((0,6,6))
    if r_list==None:
        r_list=[0 for i in range(n)]
    for i in range(n):
        G=np.zeros((6,6))
        Ixx=0.5*m_list[i]*r_list[i]**2
        Iyy=(1/12)*m_list[i]*(3*r_list[i]**2 + L_list[i]**2)
        Izz=(1/12)*m_list[i]*(3*r_list[i]**2 + L_list[i]**2)
        G = np.diag([Ixx, Iyy, Izz, m_list[i],m_list[i], m_list[i]])
        G_list=np.append(G_list,[G],axis=0)
    return G_list

def calc_S_List(L_list):
    omg_hat=[0,0,1]# screw direction
    q_list=[0,0,0]
    S_list=np.empty((0,6))
    for i in range(len(L_list)):
        L=0
        for q in range(i):
            L+=L_list[q]
        q_vec=[L,0,0]# vector from base to joint position in 0 position
        s_vec=np.append(omg_hat,q_vec)
        #q_list=np.append(q_list,q_vec,axis=0)
        S_list=np.append(S_list,[s_vec],axis=0)
    S_list=np.array(S_list).T
    return S_list

def Mi_1_i(L_list):#works only for 2D
    Mi_1_i_list=np.empty((0,4,4))
    for i in range(len(L_list)):
        M =np.eye(4)
        if i==0:
            M[0][3]=L_list[i]/2
        else:
            M[0][3]=(L_list[i]+L_list[i-1])/2
        Mi_1_i_list=np.append(Mi_1_i_list,[M],axis=0)
    M =np.eye(4)
    M[0][3]=L_list[-1]/2
    Mi_1_i_list=np.append(Mi_1_i_list,[M],axis=0)
    return Mi_1_i_list

def create_mass(L_list,m_per_c_meter=0.01):
    # m_per_meter is an scaler which shows the mass of every 1 pixel of the robot links
    # the defult value is 10 grams per pixcel
    # this value should be set by cm not pixel as the scale rate in x and y is different 
    m_list=np.array(L_list)*m_per_c_meter
    return m_list
    
#################
# the main function for reaching to next state
#################



def Model(thetalist, dthetalist, taumat, L_list):
    Slist = calc_S_List(L_list)
    m_list=create_mass(L_list) # we can ether use this function or specify the mass of each link.
    Glist=inertia_G(L_list,m_list)
    Mlist=Mi_1_i(L_list)
    g = np.array([0, 0, 0])
    Ftip = np.zeros((6))
    ddthetalist=mr.ForwardDynamics(thetalist, dthetalist, taumat, g, Ftip, Mlist,Glist, Slist)
    next_thetas, next_dthetas=mr.EulerStep(thetalist, dthetalist, ddthetalist, dt=0.1)
    return next_thetas, next_dthetas

In [50]:
thetalist = np.array([0,0,0])
dthetalist = np.array([0,0,0])
taulist = np.array([-1,-1,-1])
L=[1,1,1]
a,b=Model(thetalist ,dthetalist ,taulist,L)
print(a,b)

[0. 0. 0.] [-1.11375535  0.51404093 -0.69966683]


In [51]:
def step(self, u):
    """ given an action
        takes a step in the environment
        returns next state, reward, is_terminal, {}
    """
    # given action
    taumat = u

    # get current state
    # increment teacher traj
    self._next += self.inc

    teacher_st, learner_st = self.state  #

    # use embodiment_distance cost to compute cost of state
    c = cost(teacher_st[0], 
        teacher_st[1], 
        self.L_list, 
        learner_st[0], 
        learner_st[1], 
        self.L_list, 
        self.weights)

    # use robot_model to compute learner's next states
    next_thetas, next_dthetas = Model(learner_st[0], learner_st[1], taumat, self.L_list)

    # update learner's component of next state
    learner_stnext = next_thetas, next_dthetas
    #print(f"learner_stnext: {learner_stnext}")

    # fetch teacher's component of next state
    teacher_stnext = self.thetas[self.next], self.dthetas[self.next]
    #print(f"teacher_stnext: {teacher_stnext}")
    print(f"self.next: {self.next}")

    # update state
    #self.state = np.append(teacher_stnext, [learner_stnext],axis=0)
    self.state = np.array([teacher_stnext, learner_stnext])

    # return next state, -cost, is_terminal, extra
    return self.state, -c*10000, self.is_terminal(), {}